[Link na zadatke](http://www.fer.unizg.hr/_download/repository/vjezba_2%5B4%5D.pdf)

In [1]:
import numpy as np
import math
from common import *

norm = np.linalg.norm

### [Wrapper za brojanje fn. calls](https://stackoverflow.com/questions/1301735/counting-python-method-calls-within-another-method)

In [2]:
def counted(fn):
    def wrapper(*args, **kwargs):
        wrapper.called += 1
        return fn(*args, **kwargs)
    wrapper.called = 0
    wrapper.__name__= fn.__name__
    return wrapper

def counts(fn):
    def wrapper(f, *args, **kwargs):
        f.called = 0
        try:
            return fn(f, *args, **kwargs)
        finally:
            wrapper.calls_to_function = f.called
    wrapper.__name__= fn.__name__
    return wrapper

## Funkcije cilja

In [3]:
pt = lambda *x: np.array(x)
dim = lambda pt: len(pt)

@counted
def f1(x):
    x1, x2 = x
    return 100 * (x2 - x1 ** 2) ** 2 + (1 - x1) ** 2
f1_x0 = pt(-1.9, 2)
f1_xmin = pt(1, 1)
f1_min = 0

@counted
def f2(x):
    x1, x2 = x
    return (x1 - 4) ** 2 + 4 * (x2 - 2) ** 2
f2_x0 = pt(0.1, 0.3)
f2_xmin = pt(4, 2)
f2_min = 0

@counted
def f3(x):
    return np.sum(np.square(np.arange(1, x.shape[0] + 1) - x))
f3_x0 = pt(0, 0, 0, 0, 0)
f3_min = 0

@counted
def f4(x):
    x1, x2 = x
    return abs((x1 - x2) * (x1 + x2)) + math.sqrt(x1 ** 2 + x2 ** 2)
f4_x0 = pt(5.1, 1.1)
f4_min = 0

@counted
def f6(x):
    x = np.sum(np.square(x))
    return 0.5 + (math.sin(math.sqrt(x)) ** 2 - 0.5) / (1 + 0.001 * x) ** 2
f6_xmin = 0
f6_min = 0

### [Golden section](http://www.fer.unizg.hr/_download/repository/zlatni_rez.txt)

In [4]:
@counts
def zlatni_rez(f, a, b, e, k=0.5*(math.sqrt(5) - 1)):
    hi = b - k * (b - a)
    lo = a + k * (b - a)
    
    f_hi = f(hi)
    f_lo = f(lo)
    
    while (b - a) > e:
        if f_hi < f_lo:
            b = lo
            lo = hi
            hi = b - k * (b - a)
            f_lo = f_hi
            f_hi = f(hi)
        else:
            a = hi
            hi = lo
            lo = a + k * (b - a)
            f_hi = f_lo
            f_lo = f(lo)
    
    return (a + b) / 2

### [Unimodal](http://www.fer.unizg.hr/_download/repository/unimodalni.txt)

In [5]:
@counts
def unimodalni(f, pt, h=1):
    l, r = pt - h, pt + h
    m = pt
    step = 1
    
    fm = f(pt)
    fl = f(l)
    fr = f(r)
    
    if fm < fr and fm < fl:
        return l, r
    elif fm > fr:
        while True:
            l, m, fm = m, r, fr
            r = pt + h * (step * 2)
            fr = f(r)
            step *= 2
            if fm <= fr:
                return l, r
    else:
        while True:
            r, m, fm = m, l, fl
            l = pt - h * (step * 2)
            fl = f(l)
            step *= 2
            if fm <= fl:
                return l, r
    

### [Simplex](http://www.fer.unizg.hr/_download/repository/simplex.html)

In [6]:
class InvalidAlgorithmSetupException(Exception):
    def __init__(self, message):
        super(InvalidAlgorithmSetupException, self).__init__(message)
    
    def raiseIf(condition, message = "Invalid algorithm setup"):
        if condition:
            raise InvalidAlgorithmSetupException(message)

In [7]:
import numpy as np

In [8]:
np.mean([[1, 3], [2, 3]], axis=0)

array([ 1.5,  3. ])

In [66]:
@counts
def nelder_mead(
    f, x0, step=1, alpha=1, beta=0.5, gamma=2, sigma=0.5, epsilon=1e-6, maxIter=1000):
    InvalidAlgorithmSetupException.raiseIf(alpha <= 0)
    InvalidAlgorithmSetupException.raiseIf(gamma <= 1)
    InvalidAlgorithmSetupException.raiseIf(sigma <= 0 or sigma > 0.5)
    
    x = simplex_pts(x0, step)
    l, h = 0, -1
    
    reflect = lambda xc, xh: xc + alpha * (xc - xh)
    expand = lambda xc, xr: xc + gamma * (xr - xc)
    contract = lambda xc, xh: xc - beta * (xc - xh)
    shrink = lambda xl, xi: xl + sigma * (xi - xl)
    
    i = -1
    while True:
        i += 1
        
        x = sorted(x, key=f)
        xc = np.mean(x[:h], axis=0)
        
        if np.max(norm(x - xc)) <= epsilon or i >= maxIter:
            break
        
        xr = reflect(xc, x[h])
        
        fxr = f(xr)
        fxl = f(x[l])
        
        if fxr < fxl:
            xe = expand(xc, x[h])
            x[h] = xe if f(xe) < fxl else xr
        else:
            if all(fxr > f(xj) for xj in x[:h]):
                if fxr < f(x[h]):
                    x[h] = xr
                    
                xk = contract(xc, x[h])
                if f(xk) < f(x[h]):
                    x[h] = xk
                else:
                    shrink_xi = lambda xi: shrink(x[l], xi)
                    x = lmap(shrink_xi, x)
            else:
                x[h] = xr
            
    return x[0]

def simplex_pts(x0, step):
    ret = [x0]
    for i in range(len(x0)):
        xp = np.copy(x0)
        xp[i] += step
        ret.append(xp)
    return np.array(ret)
        

### [Hooke-Jeeves](http://www.fer.unizg.hr/_download/repository/hj.html)

* x0 - pocetna tocka
* xB - bazna tocka 
* xP - pocetna tocka pretrazivanja
* xN - tocka dobivena pretrazivanjem

In [10]:
@counts
def hooke_jeeves(f, x_0, eps=1e-6):
    dx = 0.5 * np.ones_like(x_0)
    eps = eps * np.ones_like(x_0)

    xp = np.copy(x_0)
    xb = np.copy(x_0)

    while True:
        xn = explore(f, xp, dx)

        if f(xn) < f(xb):
            xp = 2 * xn - xb
            xb = xn
        else:
            dx /= 2
            xp = xb
            if np.any(eps > dx):
                break
    return xp

def explore(f, xp, dx):
    x = np.copy(xp)
    for i in range(dim(xp)):
        P = f(x)
        x[i] = x[i] + dx[i]
        if f(x) > P:
            x[i] = x[i] - 2 * dx[i]
            if f(x) > P:
                x[i] = x[i] + dx[i]
    return x

In [11]:
hooke_jeeves(f4, pt(10, 0))

array([10,  0])

In [12]:
hooke_jeeves.calls_to_function

114

# Zadaci
## 1.
Definirajte jednodimenzijsku funkciju br. 3, koja će imati minimum u točki 3. Kao početnu točku
pretraživanja postavite točku 10. Primijenite sva tri postupka na rješavanje ove funkcije te ispišite
pronađeni minimum i broj evaluacija funkcije za svaki pojedini postupak. Probajte sve više
udaljavati početnu točku od minimuma i probajte ponovo pokrenuti navedene postupke. Što možete
zaključiti? 

In [13]:
import pandas as pd

hooke_jeeves(f3, pt(10))
hooke_jeeves.calls_to_function

data = []
for f in [ hooke_jeeves ]:
    result = f(f3, pt(10))
    data.append([f.__name__, f.calls_to_function, result])

pd.DataFrame(data, columns=['Algoritam', 'Broj poziva', 'Rezultat'])

,Algoritam,Broj poziva,Rezultat
0,hooke_jeeves,76,[10]


## 2.
Primijenite simpleks po Nelderu i Meadu, Hooke-Jeeves postupak te pretraživanje po koordinatnim
osima na funkcije 1-4 uz zadane parametre i početne točke (broj varijabli funkcije 3 najmanje 5). Za
svaki postupak i svaku funkciju odredite minimum koji su postupci pronašli i potrebni broj
evaluacija funkcije cilja koji je potreban do konvergencije (prikažite tablično). Što možete zaključiti
iz rezultata?

In [21]:
fs = [f1, f2, f3, f4]
x0s = [f1_x0, f2_x0, f3_x0, f4_x0]

data = []
for search_f in [hooke_jeeves, nelder_mead]:
    for f, x0 in zip(fs, x0s):
        result = search_f(f, x0)
        data.append(
            [search_f.__name__, x0, search_f.calls_to_function, result])

pd.DataFrame(data, columns=['Algoritam', 'x0', 'Broj poziva', 'Rezultat'])

NameError: name 'eps' is not defined

## 3.
Primijenite postupak Hooke-Jeeves i simpleks po Nelderu i Meadu na funkciju 4 uz početnu točku
(5, 5). Objasnite rezultate!

In [15]:
hooke_jeeves(f4, pt(5, 5))

array([5, 5])

In [16]:
nelder_mead(f4, pt(5, 5))

array([ 5.5,  5.5])

## 4.
Primijenite simpleks po Nelderu i Meadu na funkciju 1. Kao početnu točku postavite točku (0.5,0.5).
Provedite postupak s nekoliko različitih koraka za generiranje početnog simpleksa (primjerice iz
intervala od 1 do 20) i zabilježite potreban broj evaluacija funkcije cilja i pronađene točke
minimuma. Potom probajte kao početnu točku postaviti točku (20,20) i ponovo provesti eksperiment.
Što možete zaključiti?

## 5.
Primijenite jedan postupak optimizacije na funkciju 6 u dvije dimenzije, tako da postupak pokrećete
više puta iz slučajno odabrane početne točke u intervalu [-50,50]. Možete li odrediti vjerojatnost
pronalaženja globalnog optimuma na ovaj način? (smatramo da je algoritam locirao globalni
minimum ako je nađena vrijednost funkcije cilja manja od 10^-4)

In [17]:
ncalls = 20

data = []
for i in range(ncalls):
    random = np.random.randint(-50, 50 + 1, size=2)
    result = hooke_jeeves(f6, pt(*random))
    data.append([hooke_jeeves.__name__, 
                 random,
                 hooke_jeeves.calls_to_function, 
                 result])

pd.DataFrame(data, columns=['Algoritam', 'x0', 'Broj poziva', 'Rezultat'])

,Algoritam,x0,Broj poziva,Rezultat
0,hooke_jeeves,"[21, -33]",146,"[21, -31]"
1,hooke_jeeves,"[48, -20]",152,"[48, -15]"
2,hooke_jeeves,"[-8, -41]",157,"[-7, -40]"
3,hooke_jeeves,"[-15, 43]",151,"[-9, 43]"
4,hooke_jeeves,"[46, -48]",139,"[46, -47]"
5,hooke_jeeves,"[47, 1]",114,"[47, 1]"
6,hooke_jeeves,"[-11, -22]",134,"[-11, -22]"
7,hooke_jeeves,"[30, 20]",114,"[30, 20]"
8,hooke_jeeves,"[-40, 41]",139,"[-39, 41]"
9,hooke_jeeves,"[22, 48]",114,"[22, 48]"


# Matlab
* `fminbnd` (1 var)
* `fminsearch` (vise var; simplex po N&M)
* `fminunc` (vise var; razliciti alg.)
* `fmincon` (vise var, uz ogranicenja)